# Get the Data

In [1]:
import os
import pandas as pd
from pymongo import MongoClient
from typing import Dict, List, Any, Optional 
from datetime import datetime


In [ ]:
import json
import re

In [ ]:
# Configuration
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "digikala"
PRODUCTS_COLLECTION = "products"

class ProductDataReader:
    """
    A class to read product data from Digikala MongoDB database and convert to pandas DataFrame.
    """
    
    def __init__(self, mongo_uri: str = MONGO_URI, db_name: str = DB_NAME):
        """
        Initialize the MongoDB connection.
        
        Args:
            mongo_uri: MongoDB connection string
            db_name: Database name
        """
        self.client = MongoClient(mongo_uri)
        self.db = self.client[db_name]
        self.products_collection = self.db[PRODUCTS_COLLECTION]
        
    def get_collection_info(self) -> Dict[str, Any]:
        """
        Get basic information about the products collection.
        
        Returns:
            Dictionary containing collection statistics
        """
        try:
            total_docs = self.products_collection.count_documents({})
            sample_doc = self.products_collection.find_one()
            
            return {
                "total_documents": total_docs,
                "sample_document": sample_doc,
                "collection_name": PRODUCTS_COLLECTION,
                "database_name": DB_NAME
            }
        except Exception as e:
            print(f"Error getting collection info: {e}")
            return {}
    
    def get_specifications(self , spec_groups: List[Dict[str, Any]]) -> Dict[str, str]:
        # ---------- helpers ----------
        persian_digits = str.maketrans("۰۱۲۳۴۵۶۷۸۹", "0123456789")
        arabic_digits = str.maketrans("٠١٢٣٤٥٦٧٨٩", "0123456789")

        def to_ascii(s: Any) -> str:
            if s is None:
                return ""
            s = str(s).translate(persian_digits).translate(arabic_digits)
            return "".join(ch for ch in s if 32 <= ord(ch) <= 126)

        def join_vals(v):
            if not v:
                return ""
            if isinstance(v, (list, tuple)):
                return ", ".join([str(x) for x in v if x is not None])
            return str(v)

        def first_number(text: str) -> str:
            t = to_ascii(text)
            m = re.search(r"(\d+(?:\.\d+)?)", t)
            return m.group(1) if m else ""

        def extract_year(text: str) -> str:
            t = to_ascii(text)
            m = re.search(r"\b(20\d{2}|19\d{2})\b", t)
            return m.group(1) if m else ""

        def extract_size_3nums_mm(text: str) -> str:
            # Replace ×, X, * with x BEFORE to_ascii
            t = re.sub(r"[×X*]", "x", str(text))
            t = to_ascii(t)
            t = re.sub(r"\s*میلی[\u200c\s]*متر\s*", "", t)
            t = re.sub(r"[\u200e\u200f\u202a-\u202e]", "", t)
            nums = re.findall(r"\d+(?:\.\d+)?", t)
            if len(nums) == 3:
                return "x".join(nums)
            return ""

        def extract_inch(text: str) -> str:
            t = to_ascii(text).lower()
            m = re.search(r"(\d+(?:\.\d+)?)\s*(inch|in|'|\")", t)
            if m:
                return m.group(1)
            m = re.search(r"\b(\d+(?:\.\d+)?)\b", t)
            return m.group(1) if m else ""

        # NEW: map Persian category to high/mid/low
        def map_category(value: str) -> str:
            raw = (value or "").replace("\u200c", "").strip().lower()
            # Keep Persian for matching before ASCII stripping
            if any(k in raw for k in ["پرچم", "پرچمدار", "پرچم دار","بالا رده"]):
                return "high"
            if any(k in raw for k in ["ميان رده", "میان رده", "میانرده", "ميان‌رده", "میان‌رده"]):
                return "mid"
            if any(k in raw for k in ["پايين رده", "پایین رده", "پایینرده","اقتصادی"]):
                return "low"
            # English fallbacks
            t = to_ascii(raw)
            if "flagship" in t: return "high"
            if "mid" in t: return "mid"
            if "low" in t or "entry" in t: return "low"
            return ""

        def extract_storage_gb(text: str) -> str:
            t_raw = text or ""
            t = to_ascii(t_raw).lower()

            # TB
            m_tb = re.search(r"\b(\d{1,2})\s*(tb|tib|ترابایت|ترابايت|terabyte|terabytes)\b", t)
            if m_tb:
                gb_val = float(m_tb.group(1)) * 1024
                return str(gb_val)

            # GB
            m_gb = re.search(r"\b(\d{1,4})\s*(gb|gib|گیگابایت|گيگابايت|gigabyte|gigabytes)\b", t)
            if m_gb:
                return str(float(m_gb.group(1)))

            # MB
            m_mb = re.search(r"\b(\d{1,5})\s*(mb|mib|مگابایت|مگابايت)\b", t)
            if m_mb:
                gb_val = float(m_mb.group(1)) / 1024
                return str(round(gb_val, 3))

            # Fallback: Persian-only or number-only (if no unit found)
            n_gb = first_number(t_raw) if "گیگ" in t_raw or "g" in t else ""
            if n_gb:
                return str(float(n_gb))
            n_mb = first_number(t_raw) if "مگ" in t_raw or "m" in t else ""
            if n_mb:
                gb_val = float(n_mb) / 1024
                return str(round(gb_val, 3))
            n_tb = first_number(t_raw) if "ترا" in t_raw or "t" in t else ""
            if n_tb:
                gb_val = float(n_tb) * 1024
                return str(gb_val)

            return ""

        # ---------- flatten spec table ----------
        flat = {}
        try:
            for group in spec_groups if isinstance(spec_groups, list) else []:
                for attr in group.get("attributes", []) or []:
                    t = str(attr.get("title", "")).strip()
                    v = join_vals(attr.get("values", []))
                    if t and v and t not in flat:
                        flat[t] = v
        except Exception:
            pass

        norm = lambda s: str(s).strip().lower()  # noqa: E731
        def vby(keys: List[str]) -> str:
            keys_normed = [norm(x) for x in keys]
            for k in list(flat.keys()):
                if norm(k) in keys_normed:
                    return flat[k]
            return ""

        # ---------- outputs (edited) ----------
        out = {
            "os": "",
            "introduce_date": "",
            "category": "",
            "size": "",
            "weight": "",
            "display_technology": "",
            "refresh_rate": "",
            "size_screen_inch": "",
            "display_to_body_ratio": "",
            "pixel_per_inch": "",
            "cpu_model": "",
            "storage_gb": "",
            "ram_gb": "",
            "internet": "no",
            "camera_num": "",
            "camera_resolution_mp": "",
            "video": "",
            "battery_power_mah": "",        }

        # simple direct
        out["size"] = extract_size_3nums_mm(vby(["ابعاد", "size", "dimension", "dimensions"]))
        out["weight"] = first_number(vby(["وزن", "weight"]))

        # category mapping (high/mid/low)
        cat_raw = vby(["دسته ‌بندی", "دسته بندی", "category"])
        out["category"] = map_category(cat_raw)

        # os
        os_val = to_ascii(vby(["سیستم عامل", "os", "operating system"]))
        if not os_val:
            any_text = to_ascii(" ".join(flat.values())).lower()
            if "android" in any_text:
                os_val = "Android"
            elif "ios" in any_text or "i os" in any_text:
                os_val = "iOS"
        out["os"] = os_val

        # introduce year
        out["introduce_date"] = extract_year(vby(["تاریخ معرفی", "زمان معرفی", "introduce date", "introduction date"]))

        # display
        out["display_technology"] = to_ascii(vby(["فناوری صفحه‌ نمایش", "فناوری صفحه نمایش", "display technology", "panel", "فناوری نمایش"]))
        out["refresh_rate"] = first_number(vby(["نرخ به‌روزرسانی تصویر", "نرخ بروزرسانی", "refresh rate"]))
        out["size_screen_inch"] = extract_inch(vby(["اندازه", "اندازه صفحه", "اندازه صفحه نمایش", "display size"]))
        dbr_txt = vby(["نسبت صفحه‌ نمایش به بدنه", "نسبت نمایشگر به بدنه", "screen-to-body ratio", "display to body ratio"])
        out["display_to_body_ratio"] = first_number(dbr_txt)
        out["pixel_per_inch"] = first_number(vby(["تراکم پیکسلی", "ppi", "pixel density"]))

        # CPU/GPU
        out["cpu_model"] = to_ascii(vby(["تراشه", "چیپست", "chipset", "soc", "پردازنده", "cpu"]))


        # Memory (robust)
        out["storage_gb"] = extract_storage_gb(vby(["حافظه داخلی", "storage", "internal storage"]))
        out["ram_gb"] = extract_storage_gb(vby(["مقدار RAM", "ram", "حافظه رم"]))

      
        # Networks
        nets_txt = " ".join([
            vby(["شبکه‌های مخابراتی", "network", "networks"]),
            vby(["شبکه‌های ارتباطی قابل پشتیبانی", "communication networks"]),
        ])
        def highest_network(text: str) -> str:
            t = to_ascii(text).lower()
            if "5g" in t:
                return "5G"
            if "4g" in t or "lte" in t:
                return "4G"
            if "3g" in t:
                return "3G"
            if "2g" in t:
                return "2G"
            return "no"
        out["internet"] = highest_network(nets_txt)

        # Cameras
        out["camera_num"] = first_number(vby(["تعداد دوربین‌های پشت گوشی", "تعداد دوربین های پشت گوشی", "rear cameras", "number of rear cameras"]))
        out["camera_resolution_mp"] = first_number(vby(["رزولوشن دوربین اصلی", "دوربین اصلی", "main camera resolution"]))

        # Video capability: keep only the single highest resolution@fps found
        video_text = to_ascii(vby(["سایر مشخصات فیلمبرداری", "کیفیت فیلمبرداری", "video", "video recording"]))
        # Keep a spaced version for windowed searches; also a compact version for @fps patterns
        t_sp = video_text.lower().replace("×", "x")
        t_cp = t_sp.replace(" ", "")
        
        # Extract candidates like 8k@60fps, 4k@30fps, 1080p@60fps, 720p@240fps, etc.
        # Special handling: if fps appears as a slash-list (e.g., 30/60fps), choose the MIN (e.g., 30)
        res_tokens = ["8k","6k","5k","4k","4320p","2160p","1440p","1080p","720p","480p"]
        res_rank = {
            "8k": 4320, "6k": 3160, "5k": 2880, "4k": 2160,
            "4320p": 4320, "2160p": 2160, "1440p": 1440,
            "1080p": 1080, "720p": 720, "480p": 480,
        }
        # 1) First capture slash lists and prefer their MIN fps per resolution
        res_to_fps = {}
        slash_patterns = [
            r"(?P<res>(8k|6k|5k|4k))@(?P<fpslist>\d{1,3}(?:/\d{1,3})+)fps",
            r"(?P<res>(4320p|2160p|1440p|1080p|720p|480p))@(?P<fpslist>\d{1,3}(?:/\d{1,3})+)fps",
            r"(?P<res>(8k|6k|5k|4k))\s*\(?(?P<fpslist>\d{1,3}(?:/\d{1,3})+)fps\)?",
            r"(?P<res>(4320p|2160p|1440p|1080p|720p|480p))\s*\(?(?P<fpslist>\d{1,3}(?:/\d{1,3})+)fps\)?",
        ]
        for pat in slash_patterns:
            for m in re.finditer(pat, t_cp):
                res = m.group("res").lower()
                fps_vals = [int(x) for x in m.group("fpslist").split("/") if x]
                if fps_vals:
                    fps_min = min(fps_vals)
                    # store min fps for this res; keep the smallest if multiple slash lists exist
                    if res not in res_to_fps or fps_min < res_to_fps[res]:
                        res_to_fps[res] = fps_min
        
        # 2) Then capture single-fps patterns (only if no slash list decided for that res)
        single_patterns = [
            r"(?P<res>(8k|6k|5k|4k))@(?P<fps>\d{1,3})fps",
            r"(?P<res>(4320p|2160p|1440p|1080p|720p|480p))@(?P<fps>\d{1,3})fps",
            r"(?P<res>(8k|6k|5k|4k))\s*\(?(?P<fps>\d{1,3})fps\)?",
            r"(?P<res>(4320p|2160p|1440p|1080p|720p|480p))\s*\(?(?P<fps>\d{1,3})fps\)?",
        ]
        for pat in single_patterns:
            for m in re.finditer(pat, t_cp):
                res = m.group("res").lower()
                fps = int(m.group("fps"))
                if res not in res_to_fps:
                    res_to_fps[res] = fps
                else:
                    # without slash context, keep the max single fps seen
                    res_to_fps[res] = max(res_to_fps[res], fps)
        
        # 3) Persian-format fallback: after a resolution token, take the next number as fps (if not set yet)
        for rt in res_tokens:
            for m in re.finditer(rf"{rt}", t_sp):
                if rt in res_to_fps:
                    continue
                window = t_sp[m.end(): m.end()+60]
                mnum = re.search(r"(\d{1,3})", window)
                if mnum:
                    try:
                        res_to_fps[rt] = int(mnum.group(1))
                    except Exception:
                        pass
        
        # Choose best by resolution rank, then fps
        if res_to_fps:
            best_res = max(res_to_fps.keys(), key=lambda r: (res_rank.get(r, 0), res_to_fps[r]))
            best_fps = res_to_fps[best_res]
            label = best_res.upper() if best_res.endswith("k") else best_res
            out["video"] = f"{label}@{best_fps}FPS"
        else:
            out["video"] = ""



        # Battery / charging
        out["battery_power_mah"] = first_number(vby(["ظرفیت باتری", "battery capacity"]) or vby(["مشخصات باتری"]))
        # charging_power_w removed; keep if needed as helper:
        # out["charging_power_w"] = watt_number_max(bat_specs)

        # final ASCII clean + field trims
        for k, v in list(out.items()):
            v = to_ascii(v)
            if k in ["internet"]:
                v = v.replace("Lte", "4G")
            out[k] = v.strip()

        return out
    
    
    def process_colors(self, colors: List[str]) -> int:
        """
        Process colors list into a count of available colors.
        
        Args:
            colors: List of color strings
            
        Returns:
            Number of colors available (color diversity)
        """
        if not colors:
            return 0
        return len(colors)
    
    def process_suggestions(self, suggestions: Dict) -> Dict[str, float]:
        """
        Process suggestions dictionary into count and percentage features.
        
        Args:
            suggestions: Suggestions dictionary with 'count' and 'percentage' keys
            
        Returns:
            Dictionary with 'suggestions_count' and 'suggestions_percentage' features
        """
        if not suggestions:
            return {"suggestions_count": 0.0, "suggestions_percentage": 0.0}
        
        count = suggestions.get("count", 0)
        percentage = suggestions.get("percentage", 0.0)
        
        return {
            "suggestions_count": float(count),
            "suggestions_percentage": float(percentage)
        }
    

    
    def read_products_to_dataframe(self, 
                                 limit: Optional[int] = None,
                                 filter_query: Optional[Dict] = None,
                                 include_specifications: bool = True) -> pd.DataFrame:
        """
        Read product data from MongoDB and convert to pandas DataFrame.
        
        Args:
            limit: Maximum number of documents to retrieve (None for all)
            filter_query: MongoDB filter query to apply
            include_specifications: Whether to flatten and include specifications
            
        Returns:
            pandas DataFrame containing product data
        """
        try:
            # Build query
            query = filter_query or {}
            
            # Get cursor
            cursor = self.products_collection.find(query)
            if limit:
                cursor = cursor.limit(limit)
            
            # Convert to list
            documents = list(cursor)
            
            if not documents:
                print("No documents found matching the criteria.")
                return pd.DataFrame()
            
            print(f"Retrieved {len(documents)} documents from MongoDB.")
            
            # Process documents
            processed_docs = []
            
            for doc in documents:
                # Basic fields
                processed_doc = {
                    "brand": doc.get("brand"),
                    "category": doc.get("category"),
                    "price": doc.get("price"),
                    "rate": doc.get("rate"),
                    "count_raters": doc.get("count_raters"),
                    "popularity": doc.get("popularity"),
                    "num_questions": doc.get("num_questions"),
                    "num_comments": doc.get("num_comments"),
                }
                
                # Process complex fields
                processed_doc["color_diversity"] = self.process_colors(doc.get("colors", []))
                # Process suggestions into separate features
                suggestions_data = self.process_suggestions(doc.get("suggestions", {}))
                processed_doc.update(suggestions_data)
                
                
                # Process specifications if requested
                if include_specifications:
                    flattened_specs = self.get_specifications(doc.get("specifications", []))
                    processed_doc.update(flattened_specs)
                
                processed_docs.append(processed_doc)
            
            # Create DataFrame
            df = pd.DataFrame(processed_docs)
            
            # No need to separate features into numeric or other categories
            
            print(f"Created DataFrame with shape: {df.shape}")
            print(f"Columns: {list(df.columns)}")
            
            return df
            
        except Exception as e:
            print(f"Error reading data from MongoDB: {e}")
            return pd.DataFrame()
    
    def get_data_summary(self, df: pd.DataFrame) -> Dict[str, Any]:
        """
        Get summary statistics of the DataFrame.
        
        Args:
            df: pandas DataFrame
            
        Returns:
            Dictionary containing summary statistics
        """
        if df.empty:
            return {"error": "DataFrame is empty"}
        
        summary = {
            "shape": df.shape,
            "columns": list(df.columns),
            "missing_values": df.isnull().sum().to_dict(),
            "data_types": df.dtypes.to_dict(),
            "numeric_summary": df.describe().to_dict() if not df.select_dtypes(include=['number']).empty else {},
            "categorical_summary": {}
        }
        
        # Categorical columns summary
        categorical_cols = df.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            if col in df.columns:
                summary["categorical_summary"][col] = {
                    "unique_values": df[col].nunique(),
                    "most_common": df[col].value_counts().head().to_dict()
                }
        
        return summary
    
    def close_connection(self):
        """Close MongoDB connection."""
        if self.client:
            self.client.close()
            print("MongoDB connection closed.")


reader = ProductDataReader()

try:
   # Get collection info
    print("=== Collection Information ===")
    info = reader.get_collection_info()
    print(f"Total documents: {info.get('total_documents', 'Unknown')}")
    print(f"Database: {info.get('database_name', 'Unknown')}")
    print(f"Collection: {info.get('collection_name', 'Unknown')}")
    

    print("=== 5 Random Sample Specifications ===")
    pipeline = [
        {"$match": {}},
        {"$sample": {"size":2}},
        {"$project": {"_id": 1, "title_en": 1, "title_fa": 1, "specifications": 1}},
    ]
    for i, doc in enumerate(reader.products_collection.aggregate(pipeline), 1):
        print(f"\n--- Sample #{i} ---")
        print(f"_id: {doc.get('_id')}")
        print(f"title_en: {doc.get('title_en') or ''}")
        print(f"title_fa: {doc.get('title_fa') or ''}")

        # Original specification data (as stored)
        print("Original specifications:")
        print(json.dumps(doc.get("specifications", []), indent=2, ensure_ascii=False))

        spec = reader.get_specifications(doc.get("specifications", []))
        print("Normalized specifications:")
        print(json.dumps(spec, indent=2, ensure_ascii=True))

    
    print("=== Reading Product Data ===")
    df = reader.read_products_to_dataframe()  # Limit for testing
    
    if not df.empty:
        # # Display basic info
        # print(f"DataFrame shape: {df.shape}")
        # print(f"Columns: {list(df.columns)}")
        
        # # Get summary
        # print("=== Data Summary ===")
        # summary = reader.get_data_summary(df)

        
        # Save to CSV (optional)
        output_file = f"digikala_products_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"Data saved to: {output_file}")
        
    else:
        print("No data retrieved.")
        
except Exception as e:
    print(f"Error: {e}")

finally:
    # Close connection
    reader.close_connection()


=== Collection Information ===
Total documents: 508
Database: digikala
Collection: products
=== 5 Random Sample Specifications ===

--- Sample #1 ---
_id: 218659
title_en: BlackBerry DTEK60 BBA100-2 Mobile Phone
title_fa: گوشی موبایل بلک بری مدل DTEK60 BBA100-2
Original specifications:
[
  {
    "title": "مشخصات کلی",
    "attributes": [
      {
        "title": "ابعاد",
        "values": [
          "7 × 75.4 × 153.9 میلی‌متر"
        ]
      },
      {
        "title": "وزن",
        "values": [
          "165 گرم"
        ]
      },
      {
        "title": "توضیحات بدنه",
        "values": [
          "شیشه و فلز\r\nمجهز به حس‌گر اثر انگشت (Fingerprint Sensor) "
        ]
      },
      {
        "title": "تعداد سیم کارت",
        "values": [
          "یک عدد "
        ]
      },
      {
        "title": "نوع سیم کارت",
        "values": [
          "سایز نانو (8.8 × 12.3 میلی‌متر) "
        ]
      }
    ]
  },
  {
    "title": "صفحه نمایش",
    "attributes": [
      {
        "t

In [ ]:
import pandas as pd

def get_dataframe_from_csv(csv_file):
    """
    Reads a CSV file and returns a pandas DataFrame.
    
    Args:
        csv_file (str): Path to the CSV file.
        
    Returns:
        pd.DataFrame: DataFrame containing the CSV data.
    """
    try:
        df = pd.read_csv(csv_file)
        return df
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return pd.DataFrame()



In [69]:
df = get_dataframe_from_csv("digikala_products_20251010_091958.csv")
df_org = get_dataframe_from_csv("digikala_products_20251010_091958.csv")

In [71]:
df[::12]

,brand,category,price,rate,count_raters,popularity,num_questions,num_comments,color_diversity,suggestions_count,...,pixel_per_inch,cpu_model,storage_gb,ram_gb,internet,camera_num,camera_resolution_mp,video,battery_power_mah,selfie_resolution_mp
0,general-luxe-it,low,16720000.0,76.712329,146,2,35,103,1,50.0,...,NaN,NaN,0.004,NaN,2G,NaN,NaN,NaN,1020.0,NaN
12,apple,high,804900000.0,75.384615,13,2,12,11,2,0.0,...,460.0,Apple A15 Bionic Chipset,128.000,6.000,5G,2.0,12.0,4K@24FPS,3279.0,NaN
24,apple,high,NaN,93.966480,179,4,113,105,0,0.0,...,460.0,Apple A18 (3 nm),256.000,8.000,5G,2.0,48.0,4K@60FPS,3561.0,12.0
36,realme,low,93000000.0,86.424870,386,3,134,268,2,10.0,...,260.0,Unisoc Tiger T612 (12 nm),64.000,3.000,4G,1.0,13.0,1080p@30FPS,5000.0,5.0
48,blackview,mid,118300000.0,80.909091,22,3,31,20,2,0.0,...,260.0,Unisoc Tiger T616 (12 nm),256.000,8.000,4G,2.0,50.0,1080p@30FPS,6000.0,NaN
60,blackview,mid,NaN,88.888889,18,0,11,16,0,0.0,...,269.0,Unisoc Tiger T606,128.000,6.000,4G,2.0,13.0,NaN,5180.0,NaN
72,nothing,high,319490000.0,93.760540,593,4,397,473,2,70.0,...,394.0,Mediatek Dimensity 7200 (4 nm),256.000,12.000,5G,2.0,50.0,4K@30FPS,5000.0,32.0
84,ruggear,NaN,NaN,83.076923,13,0,14,15,0,0.0,...,NaN,Qualcomm Snapdragon 430 Chipset,32.000,3.000,4G,1.0,12.0,NaN,4000.0,NaN
96,glx,NaN,NaN,90.000000,2,0,0,2,0,0.0,...,NaN,NaN,NaN,NaN,no,1.0,NaN,NaN,NaN,NaN
108,glx,NaN,NaN,60.000000,3,0,0,2,0,0.0,...,NaN,NaN,NaN,NaN,no,1.0,NaN,NaN,NaN,NaN


In [72]:
len(df)

508

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   brand                   508 non-null    object 
 1   category                392 non-null    object 
 2   price                   286 non-null    float64
 3   rate                    508 non-null    float64
 4   count_raters            508 non-null    int64  
 5   popularity              508 non-null    int64  
 6   num_questions           508 non-null    int64  
 7   num_comments            508 non-null    int64  
 8   color_diversity         508 non-null    int64  
 9   suggestions_count       508 non-null    float64
 10  suggestions_percentage  508 non-null    float64
 11  os                      394 non-null    object 
 12  introduce_date          341 non-null    float64
 13  size                    482 non-null    object 
 14  weight                  478 non-null    fl

In [8]:
del df["fast_charge"]

# Feature Engineering

In [9]:
# Pridect category based on not NaN data
# Deleting Model feature ?
# More attention data that has price 
# Combining some features like cammera features , Display features
# Problem with TB in storage 
# Handeling text and categorical data
# Anti-water -> IP code 
# All samples with color_diversity 0 -> 1

## Handling Missing values

#### Category feature

* cpu or interner or os  (no) ram or storage

In [10]:
num_nan_os = df["category"].isna().sum()
print(num_nan_os)

116


* if phone dosnt have cpu and os and (ram or storage) or internet

In [11]:
# Ensure the columns exist; then assign 'low' to category when any condition is met
mask = (
    df["cpu_model"].isna() |
    df["ram_gb"].isna() |
    df["storage_gb"].isna() |
    df["os"].isna() |
    (df["price"] < 50_000_000) |
    (df["internet"].astype(str).str.strip().str.lower() == "no")
)

df.loc[mask, "category"] = "low"

In [12]:
num_nan_os = df["category"].isna().sum()
print(num_nan_os)

71


#### Fill with Unknown

In [13]:
df['category'] = df['category'].fillna('unknown')


In [14]:
df["category"].unique()

array(['low', 'high', 'mid', 'unknown'], dtype=object)

#### color 

In [15]:
df["color_diversity"].unique()

array([ 1,  0,  2,  4,  3,  8,  7, 13,  6,  5])

In [16]:
# Ensure the columns exist; then assign 'low' to category when any condition is met
mask = (
    (df["color_diversity"] == 0)
)

df.loc[mask, "color_diversity"] = 1

In [17]:
df["color_diversity"].unique()

array([ 1,  2,  4,  3,  8,  7, 13,  6,  5])

#### os

In [18]:
df["os"] = df["os"].fillna("unknown")

In [19]:
df["os"].unique()

array(['unknown', 'Android', 'iOS', 'BlackBerry OS'], dtype=object)

#### introduce_date

In [20]:
df["introduce_date"].unique()

array([  nan, 2024., 2021., 2022., 2025., 2023., 2020., 2017.])

In [21]:
df["introduce_date"] = df["introduce_date"].fillna("unknown")

In [22]:
df["introduce_date"].unique()

array(['unknown', 2024.0, 2021.0, 2022.0, 2025.0, 2023.0, 2020.0, 2017.0],
      dtype=object)

#### size

In [23]:
df["size"]

0             15x50x130
1             15x47x110
2             15x50x116
3         13.5x50.2x118
4       13.8x53.5x120.8
             ...       
503      142.8x70.8x9.2
504     168.4x77.2x9.05
505     168.4x77.2x9.05
506    164.44x75.78x8.9
507      164.4x75.7x8.9
Name: size, Length: 508, dtype: object

In [24]:

# Count the samples with 3 values (which have 2 'x' delimiters)
count_three_values = df['size'].str.count('x')
count_three_values_samples = (count_three_values == 2).sum()

# Count the samples with 2 values (which have 1 'x' delimiter)
count_two_values_samples = (count_three_values == 1).sum()

# 3. Print the results
print(f"Count of samples with 3 size values (e.g., 162.9x78.2x7.4): {count_three_values_samples}")
print(f"Count of samples with 2 size values (e.g., 8.877x8168): {count_two_values_samples}")

Count of samples with 3 size values (e.g., 162.9x78.2x7.4): 482
Count of samples with 2 size values (e.g., 8.877x8168): 0


#### Weight

In [25]:
df["weight"].isna().sum()

np.int64(30)

In [26]:
weight_average = (df["weight"].sum() / len(df)).round(1)

In [27]:
df["weight"].fillna(weight_average)

0       90.0
1       70.0
2       70.0
3       82.6
4       83.6
       ...  
503    162.7
504    162.7
505    162.7
506    162.7
507    162.7
Name: weight, Length: 508, dtype: float64

#### Display tech

In [28]:
df["display_technology"].unique()

array([nan, 'TFT', 'LTPO Super Retina XDR OLED', 'Super Retina XDR OLED',
       'IPS LCD', 'IPS', 'AMOLED', 'Super AMOLED', 'Foldable OLED', 'LCD',
       'OLED', 'LTPO OLED', 'LTPS', 'Foldable LTPO AMOLED', 'PLS TFT',
       'LTPO AMOLED', 'TN', 'P-OLED', 'Fluid AMOLED',
       'LTPO2 Fluid AMOLED', 'LTPO3 Fluid AMOLED',
       'Dynamic LTPO AMOLED 2X', 'Dynamic AMOLED 2X', 'PLS',
       'Super AMOLED Plus', 'Super LCD', 'LED'], dtype=object)

In [29]:
import numpy as np

# --- Ranking Groups (1 is Best, 7 is Worst) ---
# Rank 1: LTPO OLED / AMOLED 2X Variants (Best)
rank_1 = [
    'LTPO OLED', 'LTPO Super Retina XDR OLED', 'LTPO2 Fluid AMOLED', 'LTPO AMOLED',
    'Foldable Dynamic LTPO AMOLED 2X', 'LTPO AMOLED 2X', 'Dynamic LTPO AMOLED 2X'
]
# Rank 2: Premium Fixed-Rate OLED Variants
rank_2 = [
    'Super Retina XDR OLED', 'Dynamic AMOLED 2X', 'Super AMOLED Plus',
    'Dynamic AMOLED', 'Fluid AMOLED', 'Super Retina'
]
# Rank 3: Standard/Flexible OLED
rank_3 = [
    'AMOLED', 'Super AMOLED', 'OLED', 'P-OLED', 'Foldable OLED'
]
# Rank 4: Premium IPS LCD / LTPS Variants
rank_4 = [
    'LTPS IPS', 'IPS-NEO', 'a-Si AHVA', 'Retina IPS LCD', 'Liquid Retina',
    'LTPS', 'IPS Plus', 'Super IPS Plus', 'S-IPS'
]
# Rank 5: General High-Quality LCD
rank_5 = [
    'IPS LCD', 'IPS', 'Super LCD', 'PLS', 'PLS IPS', 'IGZO',
    'Super LCD 3', 'Super LCD 5', 'Super LCD 2'
]
# Rank 6: Basic LCD / TFT
rank_6 = [
    'LCD', 'TFT', 'PLS TFT'
]
# Rank 7: Twisted Nematic (Worst)
rank_7 = [
    'TN'
]

display_tech_rank_map = {}

# Assign ranks
for tech in rank_1:
    display_tech_rank_map[tech] = 1
for tech in rank_2:
    display_tech_rank_map[tech] = 2
for tech in rank_3:
    display_tech_rank_map[tech] = 3
for tech in rank_4:
    display_tech_rank_map[tech] = 4
for tech in rank_5:
    display_tech_rank_map[tech] = 5
for tech in rank_6:
    display_tech_rank_map[tech] = 6
for tech in rank_7:
    display_tech_rank_map[tech] = 7

# Assign Rank 0 for missing values
display_tech_rank_map[np.nan] = 0

##--- Apply this map to your DataFrame column (assuming your column is named 'Display_Tech') ---
df['Display_Rank'] = df["display_technology"].map(display_tech_rank_map)

In [30]:
del df["display_technology"]

#### Display to body ratio

In [31]:
df["display_to_body_ratio"].describe()

count    360.000000
mean      81.435889
std       16.841778
min       16.600000
25%       84.000000
50%       86.000000
75%       87.600000
max       92.500000
Name: display_to_body_ratio, dtype: float64

##### Noise

invalid_mask = df['display_to_body_ratio'] > 100


df.loc[invalid_mask, 'display_to_body_ratio'] = np.nan


mean_ratio = df['display_to_body_ratio'].mean()

print(f"Calculated Mean Ratio (excluding NaNs): {mean_ratio:.2f}")

df.loc[invalid_mask, 'display_to_body_ratio'] = mean_ratio



In [32]:
df["display_to_body_ratio"].describe()

count    360.000000
mean      81.435889
std       16.841778
min       16.600000
25%       84.000000
50%       86.000000
75%       87.600000
max       92.500000
Name: display_to_body_ratio, dtype: float64

#### Refresh Rate

In [33]:
df["refresh_rate"].unique()

array([ nan,  20.,  60., 120.,  90.,  30., 144.,  50.,   1.])

In [34]:
df["refresh_rate"] = df["refresh_rate"].fillna("unknown")

In [35]:
df["refresh_rate"].unique()

array(['unknown', 20.0, 60.0, 120.0, 90.0, 30.0, 144.0, 50.0, 1.0],
      dtype=object)

#### Size Screen

In [36]:
df["size_screen_inch"].unique()

array([2.4 ,  nan, 1.77, 6.74, 6.1 , 6.9 , 6.7 , 6.3 , 6.72, 6.5 , 6.67,
       6.4 , 6.6 , 6.75, 5.  , 6.56, 6.58, 6.78, 6.77, 6.55, 5.99, 1.8 ,
       4.7 , 5.5 , 1.7 , 2.5 , 2.  , 6.52, 2.8 , 6.88, 6.43, 6.53, 6.73,
       6.71, 6.79, 5.7 , 6.8 , 6.  , 6.82, 4.3 , 1.5 , 6.62, 6.65, 6.61,
       5.45, 1.4 , 3.5 , 4.  , 4.5 , 4.6 , 6.2 , 7.6 , 4.2 , 5.4 , 3.1 ,
       5.2 ])

In [37]:
df["size_screen_inch"] = df["size_screen_inch"].fillna(df["size_screen_inch"].median())

#### PPI

In [38]:
df["pixel_per_inch"].unique()

array([ nan, 460., 457., 458., 260., 392., 270., 395., 411., 264., 405.,
       399., 441., 269., 403., 396., 387., 388., 402., 394., 114., 312.,
       263., 393., 267., 413., 294., 160., 167., 121., 143., 113., 111.,
       446., 409., 254., 552., 268., 374., 401., 526., 291., 116., 179.,
       444., 217., 320., 442., 525., 410., 406., 397., 510., 265., 282.,
       261., 436., 437., 266., 389., 295., 129., 165., 228., 142., 233.,
       245., 236., 316., 498., 262., 385., 271., 375., 390., 505., 513.,
       433., 355., 538., 286., 534., 540., 220., 328., 453., 424., 196.,
       278.])

In [39]:
df["pixel_per_inch"] = df["pixel_per_inch"].fillna(df["pixel_per_inch"].median())

#### CPU

In [40]:
df["cpu_model"].unique()

array([nan, 'SC9863A', 'Apple A18 (3 nm)',
       'Apple A15 Bionic (5 nm) Chipset', 'Apple A15 Bionic Chipset',
       'Apple A19 Pro (3 nm)', 'Apple A17 Bionic (3 nm) Chipset',
       'Apple A18 Pro (3 nm)', 'Unisoc Tiger T612 (12 nm)',
       'Mediatek Helio G92 Max (12 nm)',
       'MediaTek Helio G85 (12nm) Chipset',
       'Mediatek Dimensity 7050 (6 nm)',
       'Qualcomm SM7125 Snapdragon 720G (8 nm) Chipset',
       'Qualcomm SM6225 Snapdragon 685 (6 nm)',
       'Mediatek Helio G85 (12 nm)', 'Mediatek Helio G95 (12 nm) Chipset',
       'MediaTek Helio G85 (12nm) chipset', 'Unisoc Tiger T612',
       'Qualcomm SM7125 Snapdragon 720G (8 nm)', 'Unisoc T820',
       'Unisoc Tiger T616 (12 nm)', 'MediaTek Helio P20 (MT6757) Chipset',
       'Mediatek Helio G99 (6 nm)', 'Unisoc Tiger T606',
       'Unisoc T606 (12 nm)', 'Unisoc SC9863A (28 nm)',
       'Qualcomm SM7635 Snapdragon 7s Gen 3 (4 nm)',
       'Mediatek Dimensity 7300 (4 nm)',
       'Mediatek Dimensity 7350 Pro (4 nm)',

In [41]:


cpu_scores = {
    "Apple A19 Pro (3 nm)": 100,
    "Qualcomm SM8750-AB Snapdragon 8 Elite (3 nm)": 99,
    "Apple A18 Pro (3 nm)": 98,
    "Apple A18 (3 nm)" : 97 ,
    "Qualcomm Snapdragon 8 Elite (3 nm)": 98,
    "Mediatek Dimensity 9300+ (4 nm)": 97,
    "Qualcomm SM8550-AB Snapdragon 8 Gen 2 (4 nm)": 96,
    "Qualcomm Snapdragon 8 Gen 3 Chipset": 96,
    "Snapdragon 8 Gen 3 For Galaxy": 96,
    "Mediatek Dimensity 9200+ (4 nm)": 95,
    "Apple A17 Bionic (3 nm) Chipset": 95,
    "Apple A15 Bionic Chipset" : 90,
    "Apple A15 Bionic (5 nm) Chipset" : 90,   
    "Qualcomm Snapdragon 8+ Gen 1 Chipset": 92,
    "Qualcomm SM8475 Snapdragon 8+ Gen 1 Chipset": 92,
    "Qualcomm SM8450 Snapdragon 8 Gen 1 Chipset": 90,
    "Qualcomm Snapdragon 888+ 5G Chipset": 88,
    "Qualcomm SM8350 Snapdragon 888+ 5G Chipset": 88,
    "Snapdragon 888 5G Chipset": 87,
    "Qualcomm SM8350 Snapdragon 888 5G Chipset": 87,
    "Exynos 2400 (4 nm)": 87,
    "Exynos 2400e": 86,
    "Exynos 2200 (4 nm) Chipset": 85,
    "Mediatek Dimensity 8400 Ultra (4 nm)": 85,
    "Mediatek Dimensity 8050 (6nm)": 84,
    "Qualcomm SM7325-AE Snapdragon 778G+ 5G Chipset": 83,
    "MediaTek Dimensity 8100-Max Chipset": 83,
    "Mediatek Dimensity 7300 (4 nm)": 82,
    "Mediatek Dimensity 7300 Pro (4 nm)": 82,
    "Mediatek Dimensity 7350 Pro (4 nm)": 82,
    "Mediatek Dimensity 7300 Ultra (4 nm)": 82,
    "Qualcomm SM7450-AB Snapdragon 7 Gen 1 (4 nm)": 81,
    "Qualcomm SM7450-AB Snapdragon 7 Gen 1 AE (4 nm)": 81,
    "Mediatek Dimensity 7200 (4 nm)": 80,
    "Mediatek Dimensity 7200 Ultra (4 nm)": 80,
    "Qualcomm SM8250-AC Snapdragon 870 5G Chipset": 80,
    "Qualcomm SM8250 Snapdragon 865 5G Chipset": 79,
    "Qualcomm SM8735 Snapdragon 8s Gen 4 (4 nm)": 79,
    "Qualcomm SM7635 Snapdragon 7s Gen 3 (4 nm)": 78,
    "Qualcomm Snapdragon 7s Gen 2 Chipset": 77,
    "Qualcomm SM7435-AB Snapdragon 7s Gen 2 (4 nm)": 77,
    "Snapdragon 7 Gen 3": 77,
    "Mediatek Dimensity 7050 (6 nm)": 76,
    "Mediatek Dimensity 7050 (6 nm) Chipset": 76,
    "Mediatek MT6877V/TT Dimensity 7050 (6 nm)": 76,
    "MediaTek D7050": 76,
    "Mediatek Dimensity 7025 Ultra (6 nm)": 75,
    "Qualcomm SM7225 Snapdragon 750G 5G Chipset": 75,
    "Qualcomm SM6375 Snapdragon 695 5G (6 nm)": 74,
    "Mediatek Dimensity 1080 (6 nm)": 74,
    "Mediatek Dimensity 1080 (6 nm) Chipset": 74,
    "MediaTek Dimensity 7030 (6 nm)": 74,
    "Qualcomm SM6450 Snapdragon 6 Gen 1 (4 nm)": 74,
    "Qualcomm SM6475-AB Snapdragon 6 Gen 3 (4 nm)": 74,
    "Exynos 1380 (5 nm)": 74,
    "Exynos 1480": 74,
    "MediaTek Dimensity 700 Chipset": 73,
    "Mediatek Dimensity 700 (7 nm) Chipset": 73,
    "Mediatek Dimensity 6080": 73,
    "Mediatek Dimensity 6300 (6 nm)": 73,
    "Exynos 1330 (5 nm)": 73,
    "Exynos 1580 (4 nm)": 73,
    "Qualcomm SM7125 Snapdragon 720G (8 nm)": 72,
    "Qualcomm SM7125 Snapdragon 720G (8 nm) Chipset": 72,
    "Mediatek Helio G99 (6 nm)": 72,
    "Media Tek Helio G99": 72,
    "Helio G99": 72,
    "Mediatek Helio G99 Chipset": 72,
    "MediaTek Helio G99 (MT6789V/CDZA)": 72,
    "Mediatek Helio G99 Ultra (6 nm)": 72,
    "Mediatek Helio G99 Ultra": 72,
    "Mediatek Helio G100 (6 nm)": 71,
    "Mediatek Helio G100 Ultra (6 nm)": 71,
    "Mediatek Helio G95 (12 nm) Chipset": 70,
    "Mediatek Helio G95 Chipset": 70,
    "Mediatek Helio G92 Max (12 nm)": 69,
    "Mediatek Helio G91 Ultra (12 nm)": 68,
    "Mediatek Helio G85 Chipset": 65,
    "MediaTek Helio G85 (12nm) Chipset": 65,
    "Mediatek Helio G85 (12 nm)": 65,
    "Mediatek MT6769Z Helio G85 (12 nm)": 65,
    "Mediatek MT6769H Helio G88 (12 nm)": 64,
    "Media Tek Helio G88": 64,
    "Mediatek Helio G88 Chipset": 64,
    "Mediatek Helio G88 (12 nm)": 64,
    "Mediatek Helio G81 Ultra (12 nm)": 63,
    "Mediatek Helio G81 Extreme (12 nm)": 63,
    "Mediatek Helio G81": 63,
    "Helio G81 Ultra": 63,
    "Helio G81 Ultra (12 nm)": 63,
    "Mediatek Helio G80 (12 nm) Chipset": 62,
    "Qualcomm Snapdragon 680 4G (6 nm) Chipset": 62,
    "Qualcomm SM6225 Snapdragon 680 4G (6 nm)": 62,
    "Qualcomm Snapdragon 685 (6 nm)": 61,
    "Qualcomm SM6225 Snapdragon 685 (6 nm)": 61,
    "Unisoc T820": 60,
    "Unisoc T7250 (12 nm)": 55,
    "Unisoc Tiger T616 (12 nm)": 52,
    "Unisoc Tiger T615 (12nm )": 51,
    "Unisoc Tiger T612 (12 nm)": 50,
    "Unisoc Tiger T612": 50,
    "Unisoc UMS9230 (T606)": 45,
    "unisoc UMS9230(T606)": 45,
    "Unisoc Tiger T606": 45,
    "Unisoc T606 (12 nm)": 45,
    "Unisoc T606": 45,
    "Mediatek Helio G37 Chipset": 44,
    "Mediatek Helio P22 Chipset": 44,
    "MediaTek Helio G25 (12 nm) Chipset": 43,
    "Helio G25": 43,
    "Mediatek Helio G36 (12 nm)": 43,
    "Mediatek Helio G36": 43,
    "Mediatek Helio A22 (12 nm) Chipset": 42,
    "Qualcomm Snapdragon 480 5G Chipset": 42,
    "Qualcomm Snapdragon 665 Chipset": 41,
    "Qualcomm SDM450 Snapdragon 450 (14 nm) Chipset": 40,
    "Qualcomm Snapdragon 460 Chipset": 40,
    "Qualcomm SM4250 Snapdragon 460 (11 nm)": 40,
    "Qualcomm Snapdragon 430 Chipset": 35,
    "Qualcomm MSM8916 Snapdragon 430 Chipset": 35,
    "Qualcomm MSM8937 Snapdragon 430 (28 nm)": 35,
    "Unisoc SC9863A (28 nm)": 30,
    "SC9863A": 30,
    "Unisoc Sc9863A": 30,
    "Unisoc SC9863A (28nm) Chipset": 30,
    "unisoc SC9863A1 (22nm)": 30,
    "unisoc SC9863A1 (22mm)": 30,
    "Unisoc 9863A1": 30,
    "MediaTek Helio P20 (MT6757) Chipset": 30,
    "Mediatek MT6765 Helio P35 (12nm) Chipset": 30,
    "Qualcomm Snapdragon 625 Chipset": 30,
    "Qualcomm MSM8953 Snapdragon 625 Chipset": 30,
    "Mediatek 6737WA Chipset": 25,
    "MediaTek MTk6737": 25,
    "Unisoc SC7731e chipset": 25,
    "Unisoc 6531F": 25,
    "SC6531D": 25,
    "Unisoc SC6513E": 20,
    "Unisoc SC6513F": 20,
    "Mediatek MT6572M Chipset": 15,
    "Mediatek MT6572 Chipset": 15,
    "Mediatek MT6580 Chipset": 15,
    "MediaTek MT6575 Chipset": 15,
    "Mediatek MT6577 Chipset": 15,
    "MTK6577 Plus Chipset": 15,
    "Qualcomm Snapdragon 200": 15,
    "Mediatek MT6592 Chipset": 15,
    "Qualcomm MSM8917 Snapdragon 425 Chipset": 15,
    "Qualcomm MSM8230 Snapdragon 400 Chipset": 10,
    "Qualcomm Snapdragon S4 Plus Chipset": 10,
    "Qualcomm MSM8960 Snapdragon S4 Plus Chipset": 10,
    "Qualcomm MSM8960T Snapdragon S4 Pro Chipset": 10,
    "Qualcomm MSM8655 Snapdragon S2 Chipset": 5,
    "TI OMAP 4470 Chipset": 5,
    "Qualcomm MSM8992 Snapdragon 808 Chipset": 5,
    "Qualcomm MSM8996 Snapdragon 820 Chipset": 5,
    "Qualcomm MSM8974AA Snapdragon 801 Chipset": 5,
    "Qualcomm Snapdragon 801 Chipset": 5,
    "Qualcomm MSM8952 Snapdragon 617 Chipset": 5,
    "Qualcomm SDM450 Chipset": 5,
    "MediaTek MT6260": 2,
    "MediaTek MT6260A": 2,
    "Mediatek MT6260A Chipset": 2,
    "Mediatek MT6260A": 2,
    "MT6260A": 2,
    "MTK6260A": 2,
    "MT6260": 2,
    "MTK6260": 2,
    "MediaTek MTK6260": 2,
    "MTK6260M": 2,
    "MediaTek MTK6261": 2,
    "MediaTek MTK6261M": 2,
    "Mediatek MTK6261D": 2,
    "MediaTek MTK6261D": 2,
    "MTK6261": 2,
    "MTK6261M": 2,
    "MTK62610": 2,
    "MT62610": 2,
    "Mediatek MT6261D Chipset": 2,
    "MediaTek MT6261D": 2,
    "Mediatek MTK6739": 2,
    "MediaTek" : 2,
    "mediaTek" : 2 ,
    "MTK2502": 1,
    "MediaTek 2502": 1,
    "Unisoc T107 (22 nm)": 1,
    "Unisoc T107 Chipset": 1,
    "Spreadtrum 6533": 1,
     "MediaTek Helio G85 (12nm) chipset": 65,
    "Mediatek Helio G85 (12nm) Chipset": 65,
    
    # Too vague - cannot score accurately
    "mediatek": np.nan,
    
    # Likely a modem or non-APU chip
    "MediaTek D8350": np.nan,
    
    # Already exists
    "Mediatek Helio G81 Ultra": 63,
    
    # Already exists  
    "Mediatek Dimensity 7300 Ultra": 82,
    
    # Typo/incomplete name - assuming it's the 4nm version
    "Mediatek Dimensity 8400 Ultra (4 )": 85,
    
    # IoT/feature phone chipset
    "MTK6261D Chipset": 2,
    
    # Unreleased/Unconfirmed flagship
    "Mediatek Dimensity 8350 Extreme (4 nm)": 88,  # Estimated based on naming
    
    # Unisoc's better mid-range offering
    "Unisoc T760 (6 nm)": 58,
    
    # Already exists in different naming
    "Snapdragon 888 5G": 87,
    
    # High-end chip from 2021
    "MediaTek MT6893 Dimensity 1200 5G Chipset": 84,
    
    # Mid-range 5G chip
    "Qualcomm SM6350 Snapdragon 690 5G (8 nm) Chipset": 72,
    
    # Already exists
    "Qualcomm Snapdragon 8 Gen 2 Chipset": 96,
    
    # Already exists
    "Mediatek Helio G81 (12 nm)": 63,
    
    # Legacy chip
    "MTK 6589 Chipset": 10,
    
    # Older Samsung mid-range
    "Exynos 7884 Chipset": 45,
    
    # Modern Samsung mid-range
    "Exynos 1280 (5 nm) Chipset": 73,
    
    # Already exists
    "Mediatek Helio G99": 72,
    
    # Already exists
    "Exynos 2400": 87,
    
    # Already exists  
    "Exynos 2200 (4 nm)": 85
    }

def assign_cpu_scores(df, cpu_column='cpu_model', new_column='cpu_score'):
    """
    Assign CPU scores to a DataFrame based on CPU model names.
    
    Parameters:
    df: pandas DataFrame
    cpu_column: str, name of the column containing CPU models
    new_column: str, name of the new column for scores
    
    Returns:
    pandas DataFrame with CPU scores added
    """
    
    # Create a copy to avoid modifying original
    result_df = df.copy()
    
    # Map CPU models to scores, assign NaN if not found
    result_df[new_column] = result_df[cpu_column].map(cpu_scores)
    
    # Print summary of mapping results
    total_rows = len(result_df)
    mapped_rows = result_df[new_column].notna().sum()
    unmapped_rows = total_rows - mapped_rows
    
    print("CPU Score Mapping Summary:")
    print(f"Total rows: {total_rows}")
    print(f"Successfully mapped: {mapped_rows} ({mapped_rows/total_rows*100:.1f}%)")
    print(f"Not found (NaN): {unmapped_rows} ({unmapped_rows/total_rows*100:.1f}%)")
    
    # Show some examples of unmapped CPUs if any
    if unmapped_rows > 0:
        unmapped_cpus = result_df[result_df[new_column].isna()][cpu_column].unique()
        print("\nSample of unmapped CPUs:")
        for cpu in unmapped_cpus:
            print(f"  - {cpu}")
    
    return result_df



    
# Method 1: Exact matching
df_with_scores = assign_cpu_scores(df)


# Method 2: With fuzzy matching (if needed)
# df_with_fuzzy_scores = assign_cpu_scores_with_fuzzy(df)
# print(df_with_fuzzy_scores)

CPU Score Mapping Summary:
Total rows: 508
Successfully mapped: 428 (84.3%)
Not found (NaN): 80 (15.7%)

Sample of unmapped CPUs:
  - nan
  - mediatek
  - MediaTek D8350


In [42]:
df_with_scores["cpu_score"].isna().sum()

np.int64(80)

In [43]:
df["cpu_model"].isna().sum()

np.int64(77)

#### Storage

In [77]:
df["storage_gb"].unique()

array([4.000e-03, 1.600e-02, 6.400e+01, 1.280e+02, 1.024e+03, 2.560e+02,
       5.120e+02, 1.250e-01, 3.200e+01, 0.000e+00, 4.000e+00, 5.000e-01,
       1.600e+01, 8.000e+00, 8.000e-03, 3.100e-02, 2.300e-02, 6.200e-02])

In [80]:
df[df["storage_gb"].isna()]

,brand,category,price,rate,count_raters,popularity,num_questions,num_comments,color_diversity,suggestions_count,...,pixel_per_inch,cpu_model,storage_gb,ram_gb,internet,camera_num,camera_resolution_mp,video,battery_power_mah,selfie_resolution_mp


In [76]:
df["storage_gb"] = df["storage_gb"].fillna(0)

#### RAM

In [85]:
df["ram_gb"].unique()

array([0.00e+00, 1.60e-02, 4.00e+00, 8.00e+00, 6.00e+00, 1.20e+01,
       3.00e+00, 2.00e+00, 1.60e+01, 5.00e-01, 1.00e+00, 4.00e-03,
       1.25e-01, 3.10e-02, 6.20e-02, 2.30e-02, 8.00e-03, 2.50e-01,
       7.50e-01, 1.50e+00])

In [83]:
df[df["ram_gb"].isna()]

,brand,category,price,rate,count_raters,popularity,num_questions,num_comments,color_diversity,suggestions_count,...,pixel_per_inch,cpu_model,storage_gb,ram_gb,internet,camera_num,camera_resolution_mp,video,battery_power_mah,selfie_resolution_mp
0,general-luxe-it,low,16720000.0,76.712329,146,2,35,103,1,50.0,...,NaN,NaN,0.004,NaN,2G,NaN,NaN,NaN,1020.0,NaN
1,general-luxe-it,NaN,12310000.0,77.388535,628,4,59,403,1,200.0,...,NaN,NaN,0.004,NaN,2G,NaN,NaN,NaN,800.0,NaN
2,general-luxe-it,NaN,13950000.0,80.273973,146,3,23,92,1,50.0,...,NaN,NaN,0.004,NaN,2G,1.0,NaN,NaN,800.0,NaN
81,otel,low,10450000.0,80.000000,2,1,0,2,2,0.0,...,NaN,mediatek,0.000,NaN,no,1.0,NaN,NaN,1000.0,NaN
82,otel,low,13390000.0,80.000000,1,2,1,0,2,0.0,...,NaN,MediaTek,0.000,NaN,no,1.0,NaN,NaN,1800.0,NaN
85,ruggear,NaN,NaN,73.531353,303,0,32,208,0,80.0,...,114.0,NaN,0.500,NaN,2G,1.0,NaN,NaN,800.0,NaN
95,glx,low,12240000.0,81.723731,847,5,180,607,1,10.0,...,NaN,NaN,0.000,NaN,no,NaN,NaN,NaN,1650.0,NaN
96,glx,NaN,NaN,90.000000,2,0,0,2,0,0.0,...,NaN,NaN,0.000,NaN,no,1.0,NaN,NaN,NaN,NaN
100,glx,NaN,NaN,60.000000,1,0,0,1,0,0.0,...,NaN,NaN,0.000,NaN,no,1.0,NaN,NaN,NaN,NaN
101,glx,NaN,NaN,88.571429,7,0,0,6,0,0.0,...,NaN,NaN,0.000,NaN,no,1.0,NaN,NaN,NaN,NaN


In [84]:
df["ram_gb"] = df["ram_gb"].fillna(0)

#### Internet

In [48]:
df["internet"].unique()

array(['2G', '4G', '5G', 'no', '3G'], dtype=object)

#### Camera Num

In [49]:
df["camera_num"].unique()

array([nan,  1.,  2.,  4.,  3.])

In [50]:
df["camera_num"].isna().sum()

np.int64(27)

In [51]:
df[df["camera_num"].isna()]

,brand,category,price,rate,count_raters,popularity,num_questions,num_comments,color_diversity,suggestions_count,...,storage_gb,ram_gb,sd_card_support,internet,camera_num,camera_resolution_mp,video,selfie_resolution_mp,battery_power_mah,Display_Rank
0,general-luxe-it,low,16720000.0,76.712329,146,2,35,103,1,50.0,...,4.0,NaN,no,2G,NaN,NaN,NaN,NaN,1020.0,0.0
1,general-luxe-it,low,12310000.0,77.388535,628,4,59,403,1,200.0,...,4.0,NaN,no,2G,NaN,NaN,NaN,NaN,800.0,0.0
95,glx,low,12240000.0,81.723731,847,5,180,607,1,10.0,...,NaN,NaN,no,no,NaN,NaN,NaN,NaN,1650.0,0.0
102,glx,low,NaN,77.230769,130,0,13,102,1,50.0,...,NaN,NaN,no,2G,NaN,NaN,NaN,NaN,800.0,0.0
106,glx,low,17000000.0,81.479100,933,5,232,655,2,110.0,...,NaN,NaN,no,no,NaN,NaN,NaN,NaN,1600.0,0.0
107,glx,low,30990000.0,82.285714,35,2,38,20,3,0.0,...,NaN,NaN,yes,no,NaN,NaN,NaN,NaN,1650.0,0.0
121,hanofer-mo,low,10190000.0,77.575758,33,4,10,24,4,0.0,...,32.0,24.0,no,2G,NaN,NaN,NaN,NaN,1200.0,6.0
123,hanofer-mo,low,6990000.0,61.408451,142,4,21,89,7,0.0,...,32.0,32.0,no,no,NaN,NaN,NaN,NaN,800.0,0.0
127,hanofer-mo,low,10000000.0,0.000000,0,1,0,0,2,0.0,...,4.0,4.0,no,2G,NaN,NaN,NaN,NaN,800.0,6.0
130,hanofer-mo,low,12600000.0,75.294118,102,1,20,72,1,0.0,...,32.0,64.0,no,2G,NaN,NaN,NaN,NaN,1000.0,6.0


In [52]:
df["camera_num"] = df["camera_num"].fillna(0)

#### camera_resolution_mp

In [53]:
df["camera_resolution_mp"].isna().sum()

np.int64(84)

In [103]:
df[df["camera_num"] == 0 & df["camera_resolution_mp"].isna()]

,brand,category,price,rate,count_raters,popularity,num_questions,num_comments,color_diversity,suggestions_count,...,pixel_per_inch,cpu_model,storage_gb,ram_gb,internet,camera_num,camera_resolution_mp,video,battery_power_mah,selfie_resolution_mp


In [55]:
df["camera_resolution_mp"] = df["camera_resolution_mp"].fillna(0)

#### Battery


In [60]:
df["battery_power_mah"].unique()

array([1020.,  800., 5000., 3561., 3095., 3279., 4832., 4005., 4352.,
       4441., 3240., 4685., 3582., 5825., 6000., 4500., 4300., 4180.,
       5050., 5180., 4000., 5100., 4380., 4700., 5150., 1000., 1800.,
       3950., 1650., 2200., 1600., 1200., 1500., 1750., 1450., 3000.,
       2500., 5110., 5160., 5200., 5500., 5410., 5030., 7000., 5010.,
       4800., 4400., 4020., 1400.,  600.,  900., 5800., 3020., 5230.,
       6600., 5300., 1150., 4470.,  650.,  400., 1300.,  970.,  750.,
        850., 1700.,  500., 3400., 4900., 3505., 3410., 1230., 2880.,
       2100., 3450., 2610., 2800., 3030.])

In [57]:
df[df["battery_power_mah"] == 0] = df["battery_power_mah"].median()

In [58]:
df["battery_power_mah"] = df["battery_power_mah"].fillna(df["battery_power_mah"].median())

#### video 

In [98]:
df["video"].unique()

array(['unknown', '4K@60FPS', '4K@24FPS', '4K@120FPS', '1080p@30FPS',
       '1080p@60FPS', '4K@30FPS', '1440p@30FPS', '8K@24FPS', '8K@30FPS',
       '2160p@30FPS', '720p@30FPS', '480p@15FPS', '1080p@720FPS',
       '720p@480FPS', '480p@720FPS'], dtype=object)

In [97]:
df["video"] = df["video"].fillna("unknown")